## 3 labaratorinis darbas 

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 

Feedforward tinklui mum reikės apsibrėžti tam tikras funkcijas:

* Apsibrėžti aktyvavimo funkciją, šiuo atvėju panaudosime sigmoid $f(x) = \frac{1}{1-e^{-x}}$
* Apskaičiuoti aktyvavimo funkcijos išvestinę $f'(x)$
* Apsibrėžti savo klaidos/nuokrypio funkciją (loss function), panaudosime vidutinio kvadratinio nuokrypio $MSE =\sum_{i}^{n}\frac{(\hat{y}_i - y_i)^2}{n}$

Toliau galime pereiti prie **feedforward** tinklo konstravimo, jo struktūra galime matyti paveikslėlyje žemiau. Struktūra sudaro:

* Input layer, kurį sudaro du elementai, mūsų atvėju tai būtų **ūgis** ir **svoris** (jie turi būti normalizuoti)
* Hidden layer, kurį sudaro 2 neuronai
* Output, mūsų atvėju tai būtų lytis (moteris arba vyras)

Jeigu su input sluoksniu viskas kaip ir aišku, tai stabtelkim prie antrojo (hidden layer), apžvelkime detaliau kas ten vyksta.

1. Šiame sluoksnyje yra mūsų aukščiau apibrėžta aktyvavimo funkcija. $sigmoid$
2. Kiekvienas input'o elementas yra dauginamas iš atitinkamo svorio
3. Šios sandaugos yra sudedamos ir pridedamas **bias** (b)
4. Tokia suma yra "paduodama" į aktyvavimo funkcija (mūsų atvėju tai būtų sigmoid)

visa tai atrodo taip: $sigmoid\Big(x_1\cdot w_1 + x_2 \cdot w_2 + b\Big)$


In [8]:
Image(url= "https://www.researchgate.net/publication/332932489/figure/fig1/AS:756125788020736@1557285844270/Example-of-simple-neural-network-defined-by-E-x-1-x-2-y.ppm")

Kyla klausimas, kas gi ta sigmoid aktyvavimo funkcija?? (atvaizduota grafike žemiau).
Kaip matome iš grafiko, sigmoid funkcija įgyja reikšmes iš intervalo $[0; 1]$, galėtume sakyti, kad ši funkcija konvertuoja reikšmes nuo $-\infty$ iki $+\infty$ į atitinkamas iš rėžio $[0; 1]$

In [9]:
Image(url= "https://victorzhou.com/static/dd5a39500acbef371d8d791d2cd381e0/fa73e/sigmoid.webp")

Toliau pereiname prie Output sluoksnio. Čia analogiškai kaip ir praeitame sluoksnyje:

1. Kiekvienas elementas gautas iš praeito sluoksnio yra dauginamas iš atitinkamo svorio ir pridedamas bias
2. Šios sandaugos yra sumuojamos ir taip pat paduodamos į aktyvavimo funkciją.
3. Gautas rezultatas iš aktyvavimo funkcijos ir yra mūsų rezultatas, šiuo atvėju prognoze ar tai moteris ar vyras

visa tai atrodo taip: $sigmoid\Big(h_1\cdot w_5 + h_2 \cdot w_6 + b\Big)$
Gautas rezultatas yra iš režio $[0;1]$, čia reiškmė arčiau 0 reiškiasi, kad lytis yra vyras, o reikšmė arčiau 1 reiškia, kad lytis yra moteris.

Kadangi pirminiai tinklo svoriai ir bias yra parenkami atsitiktinai, jie greičiausiai bus ne patys optimaliausi.. Mes tikimes, kad output ($\hat{y}$) reikšmė bus labai arti mūsų tikrosios reikšmės ($y$), tai mes įvertinsime pagal **MSE** (paminėta aukščiau), taigi mes norime minimizuoti MSE, $MSE \longrightarrow MIN$, kad tai padaryti mums reikia keisti prieš tai paminėtus svorio ir bias parametrus, bet kaip žinoti kurio parametro reikšmę į kokią keisti, kad sumažinti mūsų paklaidą?
Atsakymas paprastas žiūrėkime į mūsų **loss** funkciją kaip į daugiamačią funkciją $L(w_1, w_2, \dots, b_1, b_2, \dots)$, tuomet reikia skaičiuot dalines išvestinę kiekvienam kintamajam.

Pažiūrėkime kaip skaičiuotume dalinę išvestinę kintamajam $w_1$:

$$\frac{\partial L}{\partial w_1} = \frac{\partial L}{\partial \hat{y}}\cdot \frac{\partial \hat{y}}{\partial w_1}$$


Mes galime apskaičiuoti $\frac{\partial L}{\partial \hat{y}}$, nes prieš tai paskaičiavome $L = (1-\hat{y})^2$, reiškiasi:

$$\frac{\partial L}{\partial \hat{y}} = \frac{\partial(1-\hat{y})^2}{\partial \hat{y}} = -2(1-\hat{y})$$

Toliau reikia išsiaiškinti su $\frac{\partial \hat{y}}{\partial w_1}$, prisimename, kad mūsų prognozė $\hat{y} = f(w_5\cdot h_1 + w_6 \cdot h_2 + b_3)$, čia *f* yra aktyvavimo funkcija sigmoid, kadangi $w_1$ įtakoja tik $h_1$, tai mes galime parašyti taip:
$$\frac{\partial\hat{y}}{\partial w_1} = \frac{\partial\hat{y}}{\partial h_1} \cdot \frac{\partial h_1}{\partial w_1}$$

O tai yra ne kas kita kaip: $\frac{\partial\hat{y}}{\partial w_1} = w_5 \cdot f'(w_5 h_1 + w_6 h_2 + b_3)$

Analogiškai $\frac{\partial h_1}{\partial w_1} = x_1 \cdot f'(w_1 x_1 + w_2 x_2 + b_1), \quad$ čia $h_1 = f(w_1x_1 + w_2+x_2 + b_1)$

Taigi iš čia seka:

$$\frac{\partial L}{\partial w_1} = \frac{\partial L}{\partial \hat{y}}\cdot \frac{\partial\hat{y}}{\partial h_1} \cdot \frac{\partial h_1}{\partial w_1}$$

Šis būdas dar kitaip vadinamas **backpropagation**

Dabar galime pereiti prie tinklo apmokymo, optimizavimui naudosime *stochastic gradient descent* (SGD), kuris pasakys mum kaip pakeisti mūsų svorius, kad minimuzuoti nuostolių funkciją, tai tikrai ne sudėtinga:

$$ w_1 \leftarrow w_1 - \eta\frac{\partial L}{\partial w_1}, \quad$$
čia $\eta$ yra mokymosi žingsnis **(learning rate)**, kuris apsprendžia kaip greitai mūsų tinklas mokinsis. Visa kita kaip ir akivaizdu:

* Jeigu $\frac{\partial L}{\partial w_1} > 0, \quad w_1$ sumažės, ko pasekoje **L** sumažės
* Jeigu $\frac{\partial L}{\partial w_1} < 0, \quad w_1$ padidės, ko pasekoje **L** sumažės

Tokiu būdų mes mažais žingsniukais minimuzuosime nuostolių funkciją ir tinkamai apmokinsime tinklą.
***
Apibendrinant trumpai pakartosime visus žingsnius:
1. Pasirenkame vieną input'ą (eilutę) iš mūsų dataset'o (imties);
2. Apskaičiuojame visas nuostolių funkcijos dalines išvestines;
3. Atnaujiname svorius;
4. Grįžtame prie 1 žingsnio.

Taigi pereikime prie praktinės daleis

#### Sudarykite neuroninį tinklą, kuris atstatytų penktojo vartotojo lyties reikšmę. Ūgio ir svorio požymiai tai neuroninio tinklo įėjimo duomenys, lyties reikšmė – išėjimo.

Užkrauname reikiamus paketus

In [7]:
!pip install ipynb
import pandas as pd
from ipynb.fs.full.network import *

Nuskaitome duomenys

In [10]:
d = pd.read_csv("/home/robert/Desktop/Laborai/nn/3lab/duomenys.csv", sep=';', index_col=False)

del d["Pacientas"]
d

,Lytis,Ūgis,Svoris
0,0,165,55
1,1,183,95
2,0,171,60
3,1,194,102
4,1,185,90


Atskiriame įvesties stulpelius nuo prognozuojamo stuleplio

In [3]:
d_arr = np.asarray(d) # paverciam i masyva
X = np.asarray(d.iloc[:,1:])
y = np.asarray(d.iloc[:,0])

Standartizuojame 

In [4]:
# standartizuojam
vid = X.mean(axis=0)
std = X.std(axis=0)

X = (X-vid)/std

Apibrėžiame mūsų tinklą ir paduodame duomenys treniruotis

In [5]:
# Train our neural network!
network = OurNeuralNetwork()
network.train(X, y)
print("\nTINKLAS APMOKINTAS!\n")

Epoch 0 loss: 0.579
Epoch 10 loss: 0.573
Epoch 20 loss: 0.563
Epoch 30 loss: 0.545
Epoch 40 loss: 0.500
Epoch 50 loss: 0.369
Epoch 60 loss: 0.213
Epoch 70 loss: 0.166
Epoch 80 loss: 0.136
Epoch 90 loss: 0.112
Epoch 100 loss: 0.093
Epoch 110 loss: 0.079
Epoch 120 loss: 0.067
Epoch 130 loss: 0.058
Epoch 140 loss: 0.051
Epoch 150 loss: 0.045
Epoch 160 loss: 0.040
Epoch 170 loss: 0.036
Epoch 180 loss: 0.033
Epoch 190 loss: 0.030
Epoch 200 loss: 0.027
Epoch 210 loss: 0.025
Epoch 220 loss: 0.024
Epoch 230 loss: 0.022
Epoch 240 loss: 0.020
Epoch 250 loss: 0.019
Epoch 260 loss: 0.018
Epoch 270 loss: 0.017
Epoch 280 loss: 0.016
Epoch 290 loss: 0.015
Epoch 300 loss: 0.014
Epoch 310 loss: 0.014
Epoch 320 loss: 0.013
Epoch 330 loss: 0.012
Epoch 340 loss: 0.012
Epoch 350 loss: 0.011
Epoch 360 loss: 0.011
Epoch 370 loss: 0.011
Epoch 380 loss: 0.010
Epoch 390 loss: 0.010
Epoch 400 loss: 0.009
Epoch 410 loss: 0.009
Epoch 420 loss: 0.009
Epoch 430 loss: 0.008
Epoch 440 loss: 0.008
Epoch 450 loss: 0.008

Apmokinę tinklą, paduodame pekntojo vartotojo ūgį ir svorį, kad atstatyti lyties reikšmę (0,1)

In [6]:
V5 = [162, 58]
# standartizuojame
V5 = (V5 - vid)/std

# paduodame i tinkla
lytis = int(round(network.feedforward(V5)))
print("Vardas 5 Lytis yra: {}".format(lytis))

Vardas 5 Lytis yra: 0
